In [1]:
import math
import pprint
import yaml
from IPython.core.display import display, HTML, Javascript
from jinja2 import Template

In [2]:
with open('colours.yaml') as fobj:
    palette = yaml.load(fobj)

pprint.pprint(palette)

{'big-stone': (20, 43, 68),
 'brink-pink': (255, 112, 142),
 'buccaneer': (88, 44, 43),
 'caper': (199, 230, 170),
 'caribbean-green': (22, 201, 141),
 'carnation': (250, 94, 91),
 'chambray': (29, 80, 141),
 'chino': (207, 203, 175),
 'falu-red': (132, 30, 27),
 'keppel': (71, 168, 153),
 'lochmara': (40, 138, 214),
 'malibu': (138, 190, 229),
 'mulberry': (191, 83, 141),
 'paris-daisy': (254, 239, 109),
 'saffron': (255, 200, 63),
 'trendy-pink': (104, 78, 121)}


In [3]:
primary_colour = palette.get('lochmara')

In [4]:
template = Template("""
<style>
    .card {
        display: inline-block;
        width: 25%;
        height: 150px;
        padding: 1rem .5rem;
    }
    
    .card-swatch {
        height: 100px;
        width: 100%;
        overflow: hidden;
        text-align: center;
        padding: 1rem
    }
    
    .card-meta {
        height: 50px;
        width: 100%;
        text-align: center;
        overflow: hidden;
    }
</style>
{% for name, colour in palette %}
<div class="card">
    <div class="card-swatch" style="background-color:rgb{{ colour }}" />
    <p class="card-meta">
        <strong>{{ name }}</strong><br/>
        <code>rgb{{ colour }}</code>
        </p>
</div>
{% endfor %}
""")

HTML(template.render(palette=palette.items()))

In [5]:
def colour_hex(rbg):
    return '#{:02x}{:02x}{:02x}'.format(*rbg)

print(colour_hex(primary_colour))

#288ad6


In [6]:
# <https://www.w3.org/TR/2008/REC-WCAG20-20081211/#relativeluminancedef>

def normalize(colour):
    colour = colour / 255

    if colour <= 0.03928:
        return colour / 12.92
    else:
        return math.pow(((colour + 0.055) / 1.055), 2.4)

def luminance(rgb):
    red, green, blue = rgb
    
    return math.sqrt(0.299 * math.pow(normalize(red), 2)+
                     0.587 * math.pow(normalize(green), 2) +
                     0.144 * math.pow(normalize(blue), 2))

print(luminance(primary_colour))

0.3211927369645976


In [7]:
def contrast(a, b):
    a = luminance(a)
    b = luminance(b)
    
    min_lum = min(a, b)
    max_lum = max(a, b)
    
    return (max_lum + 0.05) / (min_lum + 0.05)

white = (255, 255, 255)
black = (0, 0, 0)

print(contrast(white, black))
print(contrast(black, (1, 1, 1)))

21.29778313018444
1.0061609248862333


## Contrast Scoring

* **Fail** - Your text doesn't have enough contrast with the background. You probably want to make it darker. This is a score of less than `3.0`.
* **AA Large** - The smallest acceptable amount of contrast for type sizes of `18px` and larger. This is a score of at least `3.0`.
* **AA** - This is the sweet spot for text sizes below ~`18px`. This is a score of at least `4.5`.
* **AAA** - This is enhanced contrast with a score of at least `7.0`. Think longer form articles that will be read for a significant period of time.

In [8]:
def contrast_score(ratio):
    if ratio >= 7.0:
        return 'AAA'
    elif ratio >= 4.5:
        return 'AA'
    elif ratio >= 3.0:
        return 'AA Large'
    else:
        return 'Fail'

print(contrast_score(contrast(white, black)))
print(contrast_score(contrast(black, (1, 1, 1))))

AAA
Fail


In [9]:
template = """
<div class="card">
    <div class="card-swatch" style="background-color:rgb{0};color:rgb{1}">
        Lorem ipsum dolor sit amet, consectetur adipiscing elit. In blandit ligula eros, sit amet ornare metus mattis sed. Morbi congue.
    </div>
    <p class="card-meta">{2}</p>
</div>
"""

display(HTML(''.join([
    template.format(primary_colour, white, contrast_score(contrast(primary_colour, white))),
    template.format(primary_colour, black, contrast_score(contrast(primary_colour, black)))])))

In [10]:
colours = []

for key, value in palette.items():
    colour = {
        'name': key,
        'colour': value,
        'contrasts': []
    }
    
    for k, v in palette.items():
        score = contrast_score(contrast(value, v))
        
        colour['contrasts'].append({
            'name': k,
            'colour': v,
            'score': score
        })
    
    colours.append(colour)

In [11]:
template = Template("""
<style></style>
{% for colour in colours %}
<div>
    <h3><strong>{{ colour.name }} <code>{{ colour.colour }}</code></strong></h3>
    <div class="card-group">
    {% for contrast in colour.contrasts %}
        <div class="card">
            <div class="card-swatch" style="background-color:rgb{{ colour.colour }};color:rgb{{ contrast.colour }}">
                Lorem ipsum dolor sit amet, consectetur adipiscing elit. In blandit ligula eros, sit amet ornare metus mattis sed. Morbi congue.
            </div>
            <div class="card-meta">
                <strong>{{ contrast.score }}</strong><br/>
                {{ contrast.name }} on {{ colour.name }}
            </div>
        </div>
    {% endfor %}
    </div>
</div>
{% endfor %}
""")

HTML(template.render(colours=colours))

In [21]:
from PIL import Image

def Image(name, rgb):
    img = Image.new('RGB', (150, 150), rgb)
    img.save('swatches/{}.png'.format(name), 'png')

generate_swatch('primary', primary_colour)

In [24]:
for name, rgb in palette.items():
    generate_swatch(name, rgb)